In [2]:
%load_ext autoreload
%autoreload 2

In [1]:
from erisk.utils import get_spark

spark = get_spark()
df = spark.read.csv(
    "gs://dsgt-clef-erisk-2024/task1/processed/baseline_nb_tfidf/v2/inference/predictions.csv",
    header=True,
    inferSchema=True,
)
df.printSchema()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/25 23:15:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/03/25 23:15:58 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).


root
 |-- symptom_number: integer (nullable = true)
 |-- Qo: string (nullable = true)
 |-- sentence_id: string (nullable = true)
 |-- position_in_ranking: integer (nullable = true)
 |-- score: double (nullable = true)
 |-- system_name: string (nullable = true)



In [3]:
from pyspark.sql import functions as F

text = spark.read.parquet(
    "gs://dsgt-clef-erisk-2024/task1/parquet/tfidf/v2/data"
).select(F.col("docid").alias("sentence_id"), "text")

text_with_preds = (
    df.join(text, on="sentence_id", how="inner")
    .where("position_in_ranking < 20")
    .cache()
)

In [4]:
text.sample(0.01).show(n=5, truncate=False)

+-----------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|sentence_id|text                                                                                                                                                                                                                                                                                                                                                                                                             |
+-----------+-----------------------------------------------------------------------------------------------------------------------------------------------------------

In [5]:
for row in (
    text_with_preds.groupBy("symptom_number")
    .count()
    .orderBy("symptom_number")
    .collect()
):
    print(row.symptom_number)
    (
        text_with_preds.where(F.col("symptom_number") == row.symptom_number)
        .drop("Q0", "system_name")
        .where("position_in_ranking < 5")
        .orderBy("position_in_ranking")
    ).show(n=10, vertical=True, truncate=200)

1
-RECORD 0-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 sentence_id         | 174521_5_2                                                                                                                                                                                               
 symptom_number      | 1                                                                                                                                                                                                        
 Qo                  | Q0                                                                                                                                                                                                       
 position_in_ranking | 1                                                                          